# 99 · Integración con Udacity Simulator

Orquesta el servidor `tools/sim_drive.py`:
1) Comprueba compatibilidad (modelo ↔ preprocesado).
2) Construye el comando.
3) Lanza y para el servidor desde aquí.

> Abre el simulador y entra en **Autonomous Mode** antes o después de lanzar el servidor.


In [ ]:
# %%bash
# (Opcional) instala dependencias
# pip install python-socketio eventlet flask pillow opencv-python

In [ ]:
from pathlib import Path
import sys, json, subprocess, shlex, time
from datetime import datetime

ROOT = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from src.config import load_preset
from src.models import build_model
from src.datasets import ImageTransform

# === Usuario: ajusta estos valores si quieres ===
PRESET = "accurate"
MODEL_NAME = "pilotnet_snn"      # "pilotnet_snn" | "pilotnet_ann" | "snn_vision"
CKPT = max((ROOT/"outputs").rglob("model_best.pt"), key=lambda p: p.stat().st_mtime)  # último best
PORT = 4567

print("ROOT:", ROOT)
print("CKPT seleccionado:", CKPT)

CFG = load_preset(ROOT/"configs"/"presets.yaml", PRESET)
DATA, MODEL = CFG["data"], CFG["model"]

# Defaults desde preset (puedes sobreescribir en la celda de lanzamiento)
ENCODER = DATA["encoder"]
T       = int(DATA["T"])
GAIN    = float(DATA["gain"])
W       = int(MODEL["img_w"])
H       = int(MODEL["img_h"])
TO_GRAY = bool(MODEL["to_gray"])

print(f"[preset={PRESET}] model={MODEL_NAME} {W}x{H} gray={TO_GRAY} | enc={ENCODER} T={T} gain={GAIN}")


In [ ]:
import torch

# Construye el modelo y carga el checkpoint para validar compatibilidad
tfm = ImageTransform(W, H, to_gray=TO_GRAY, crop_top=None)
model = build_model(MODEL_NAME, tfm, beta=0.9, threshold=0.5)
state = torch.load(CKPT, map_location="cpu")
if isinstance(state, dict) and "state_dict" in state:
    model.load_state_dict(state["state_dict"])
else:
    model.load_state_dict(state)
model.eval()
print("OK: modelo cargado y listo para inferencia.")


In [ ]:
# Construye el comando (puedes añadir --crop-top, --rgb, etc.)
cmd = f"""
python tools/sim_drive.py \
  --ckpt {shlex.quote(str(CKPT))} \
  --model-name {MODEL_NAME} \
  --preset accurate \
  --port {PORT} \
  --target-speed 12 \
  --crop-top 0 \
  --encoder rate \
  --T 30 \
  --gain 0.5 \
  --img-w 200 \
  --img-h 66 \
  --rgb False
""".strip()

print("Comando:\n", cmd)
proc = subprocess.Popen(cmd, shell=True, cwd=ROOT)
print("Servidor lanzado. PID =", proc.pid)


Ahora abre el **Udacity Simulator** y entra en **Autonomous Mode**.
Deberías ver cómo el coche comienza a moverse. Vuelve aquí para ver logs o parar el servidor.


In [ ]:
import os, signal, time

if proc and (proc.poll() is None):
    print(f"Servidor VIVO (PID={proc.pid})")
else:
    print("Servidor NO está vivo.")

# Para detener:
# if proc and (proc.poll() is None):
#     os.kill(proc.pid, signal.SIGTERM)  # en Windows: proc.terminate()
#     time.sleep(1.0)
#     print("Servidor detenido.")


## Notas
- Este notebook solo orquesta la ejecución de `tools/sim_drive.py`.
- Si tienes errores de compatibilidad de dimensiones, revisa `img_w/img_h`, `to_gray` y `--crop-top`.
- Para curvas pronunciadas, baja `--target-speed` y ajusta `--kp/--kd`.
